Drive Mounth

In [ ]:
#colab
# google-drive-ocamlfuseのインストール
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Colab用のAuth token作成
from google.colab import auth
auth.authenticate_user()

# Drive FUSE library用のcredential生成
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# drive/ を作り、そこにGoogle Driveをマウントする
!mkdir -p drive
!google-drive-ocamlfuse drive

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [ ]:
from google.colab import files
!pip install -U -q PyDrive

import tensorflow as tf
import timeit

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
drive = ""
#login function
def drive_login():
  # Authenticate and create the PyDrive client.
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  return drive


Install and import Libraries

In [ ]:
!pip install git+https://github.com/librosa/librosa
!pip install np_utils
!pip install keras
!pip install pyeasyga

import librosa
import numpy as np
import pandas as pd
import random
import time
import glob
import os
import tensorflow as tf
import np_utils
import pyeasyga

from pyeasyga import pyeasyga
from os import listdir
from os.path import isfile, join

from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import MeanShift
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

**Load Data**

In [ ]:
def load_sound_files(file_paths):
    raw_sounds = []
    for fp in file_paths:
        fp="drive/app/TUDiplomna/audio/"+fp
        X,sr = librosa.load(fp)
        raw_sounds.append(X)
    return raw_sounds

start = time.time()
mypath="drive/app/TUDiplomna/audio/"
labelsN=["Acoustic_guitar", "Applause", "Bark", "Bass_drum", "Burping_or_eructation", "Bus", "Cello", "Chime", "Clarinet", "Computer_keyboard"]
onlyfiles = []
tr_labels=pd.read_csv("drive/app/TUDiplomna/train.csv" , error_bad_lines=False)
for f in range(len(tr_labels)):
  if tr_labels.iloc[f].label in labelsN:
    onlyfiles.append(tr_labels.iloc[f].fname)
end = time.time()
print("Time for data loading:")
print(end - start)
print("seconds")
random.shuffle(onlyfiles,random.random) #mix files
train_files= onlyfiles[:500] 
test_files= onlyfiles[500:600]

#Feature Extraction

In [ ]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    #tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast

def parse_audio_files(files_names):
    features, labels = np.empty((0,187)), np.empty(0)
    counter=0
    for fn in files_names:
      try:
        mfccs, chroma, mel, contrast = extract_feature("drive/app/TUDiplomna/audio/"+fn)
      except Exception as e:
        counter=counter+1
        continue
      ext_features = np.hstack([mfccs,chroma,mel,contrast])
      features = np.vstack([features,ext_features])
      labels = np.append(labels, fn)
    #print(counter)
    return features,labels

In [ ]:
def lablesExtraction(files,labels):
  labelsSet=[]
  for i in range(0, len(files)):
    for j in range(0, 9472):#check where this num come from
      if files[i]==labels['fname'].values[j]:
        label=labels['label'].values[j]
        if label != "Acoustic_guitar" and label != "Applause" and label != "Bark" and label != "Bass_drum" and label != "Bus":
          labelsSet.append('None')
        else:
          labelsSet.append(label)
        break
  return labelsSet

def oneHotEncodder(labels):
  encoder = LabelEncoder()
  encoder.fit(labels)
  encoded_Y = encoder.transform(labels)
  return encoded_Y


In [ ]:
start = time.time()

tr_features, tr_files= parse_audio_files(train_files)
ts_features, ts_files = parse_audio_files(test_files)

X_train=pd.DataFrame(tr_features)
labels_train=lablesExtraction(tr_files,tr_labels)
Y_train=oneHotEncodder(labels_train)
#pd.DataFrame(tr_labels)

X_test=pd.DataFrame(ts_features)
labels_test=lablesExtraction(ts_files,tr_labels)
Y_test=oneHotEncodder(labels_test)


end = time.time()
print("Time for extracting test and train features and labels:")
print(end - start)
print("seconds")

In [ ]:
def makeArrayCategorial(y_data):
  dummy_y = np_utils.to_categorical(y_data)
  return dummy_y

y_train_cat=makeArrayCategorial(Y_train)
y_test_cat=makeArrayCategorial(Y_test)

#**Classification**


In [ ]:
import tensorflow as tf
tf.reset_default_graph()

*One layer perceptron*

In [ ]:
start = time.time()
# create model -Baseline: 53.72% (15.21%)
model = Sequential()
model.add(Dense(187, input_dim=187,init='uniform', activation='relu'))
model.add(Dense(6, activation='softmax'))
#Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X_train, y_train_cat, epochs=150, batch_size=10)
# evaluate the model
scores = model.evaluate(X_test, y_test_cat, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
end = time.time()
print("Time for executing  one layered perceptron")
print(end - start)
print("seconds")

*Multilayer  perceptron*

In [ ]:
start = time.time()
model = Sequential()
model.add(Dense(18, input_dim=187, init='uniform', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(18, init='uniform', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6, init='uniform', activation='softmax'))
#Compile model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# Fit the model
model.fit(X_train, y_train_cat, epochs=150, batch_size=10, verbose=0)
# evaluate the model
scores = model.evaluate(X_test, y_test_cat, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
end = time.time()
print("Time for executing  two layers perceptron with drop out")
print(end - start)
print(" in seconds")

*Logistic regression*

In [ ]:
start = time.time()
# all parameters not specified are set to their defaults
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, labels_train)
# Use score method to get accuracy of model
score = logisticRegr.score(X_test,labels_test)
print("Accuracy:")
print(score)
end = time.time()
print("Time for executing linear regression model")
print(end - start)
print(" in seconds")

*Random Forest*

In [ ]:
st= time.time()
classifier = RandomForestClassifier(n_estimators=50, max_depth=50, random_state=1)
classifier.fit(X_train, Y_train)
score=classifier.score(X_test, Y_test)
print("Accuracy:")
print(score)
print ('Time for executing Random Forest: '+str(int(time.time()-st)))

*Recurent Neural Network*

In [ ]:
#LSTM
from keras.layers import TimeDistributed
from keras.layers import AveragePooling1D
from keras.layers import Input, Flatten
data_dim = 187
timesteps = 1
num_classes = 6

model = Sequential()
model.add(LSTM(187, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 30
model.add(LSTM(90, return_sequences=True))  # returns a sequence of vectors of dimension 30
model.add(LSTM(42))  # return a single vector of dimension 30
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()
X_train_LSTM = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
X_test_LSTM = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))
model.fit(X_train_LSTM, y_train_cat, batch_size = 100, epochs = 60, verbose = 0) #verbose =0 -bez history,1 s history
start = time.time()
# evaluate the model
scores = model.evaluate(X_test_LSTM, y_test_cat, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
end = time.time()
print("Time for executing  two layers perceptron with drop out")
print(end - start)
print(" in seconds")


#Clusterization

In [ ]:
X=X_train
print (X)

MeanShift

In [ ]:
start = time.time()
#https://scikit-learn.org/stable/modules/clustering.html#mean-shift
clustering = MeanShift()
cluster_labels = clustering.fit_predict(X)
print("Best cluster num is :", len(list(set(cluster_labels))))
silhouette_avg = silhouette_score(X, cluster_labels)
print("The average silhouette_score is :", silhouette_avg)
end = time.time()
print("Time for executing mean shift model")
print(end - start)
print(" in seconds")

In [ ]:
# define a fitness function -da q napisha po dobre
def fitness(individual, data):
    temp=0.77;
    cNum=1
    for x in data:
      if temp / float(3.5) > x['score']:
        temp=x['score']
        cNum=x['clusterNum']
    return cNum


KMeans

In [ ]:
start = time.time()
#KMeans(random init)
data=[];
for i in range(2,20):
  cluster_labels = KMeans(n_clusters=i, init='random', random_state=10).fit_predict(X)
  data.append({'clusterNum': i , 'score': silhouette_score(X, cluster_labels)})
  
# initialise the GA with data
ga = pyeasyga.GeneticAlgorithm(data)
ga.fitness_function = fitness               # set the GA's fitness function
ga.run()                                    # run the GA
bestSolution=ga.best_individual()           # print the GA's best solution
for x in data:
    if x['clusterNum'] == bestSolution[0]:
        silhouette_avg=x['score']
        break
print("Best cluster num is :", bestSolution[0])
print("The average silhouette_score is :", silhouette_avg)
end = time.time()
print("Time for executing Kmeans with GA")
print(end - start)
print(" in seconds")

KMeans++

In [ ]:
start = time.time()
#KMeans++
data=[];
for i in range(2,20):
  cluster_labels = KMeans(n_clusters=i, init='random').fit_predict(X)
  data.append({'clusterNum': i , 'score': silhouette_score(X, cluster_labels)})
  
# initialise the GA with data
ga = pyeasyga.GeneticAlgorithm(data)
ga.fitness_function = fitness               # set the GA's fitness function
ga.run()                                    # run the GA
bestSolution=ga.best_individual()           # print the GA's best solution
for x in data:
    if x['clusterNum'] == bestSolution[0]:
        silhouette_avg=x['score']
        break
print("Best cluster num is :", bestSolution[0])
print("The average silhouette_score is :", silhouette_avg)
end = time.time()
print("Time for executing Kmeans++ with GA")
print(end - start)
print(" in seconds")